In [1]:
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor
from sklearn.datasets import load_diabetes
import matplotlib.pyplot as plt
import numpy as np

In [2]:
X, y = load_diabetes(return_X_y=True)
X.shape, y.shape

((442, 10), (442,))

In [3]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25)

In [4]:
def gb_predict(X, trees_list, eta):
    # Реализуемый алгоритм градиентного бустинга будет инициализироваться нулевыми значениями,
    # поэтому все деревья из списка trees_list уже являются дополнительными и при предсказании
    # прибавляются с шагом eta
    
#     predictions = np.zeros(X.shape[0])
#     for i, x in enumerate(X):
#         prediction = 0
#         for alg in trees_list:
#             prediction += eta * alg.predict([x])[0]
#         predictions[i] = prediction
        
    predictions = np.array(
        [sum([eta * alg.predict([x])[0] for alg in trees_list]) for x in X]
    )

    return predictions

В качестве функционала ошибки будем использовать среднеквадратичную ошибку. Реализуем соответствующую функцию.

In [5]:
def mean_squared_error(y_real, prediction):
    return (sum((y_real - prediction)**2)) / len(y_real)

Используем $L_{2}$ loss $L(y, z) = (y - z)^{2},$ ее производная по $z$ примет вид $L'(y, z) = 2(z - y)$. Реализуем ее также в виде функции (коэффициент 2 можно отбросить).

In [6]:
def residual(y, z):
    return - (z - y)

Реализуем функцию обучения градиентного бустинга.

In [7]:

def evaluate_alg(X_train, X_test, y_train, y_test, trees, eta):
    train_prediction = gb_predict(X_train, trees, eta)
    train_error = mean_squared_error(y_train, train_prediction)
    test_prediction = gb_predict(X_test, trees, eta)
    test_error = mean_squared_error(y_test, test_prediction)
    return train_error, test_error

In [8]:

def get_error_plot(train_err, test_err, x_name):
    plt.xlabel(x_name)
    plt.ylabel('MSE')
    plt.xlim(0, len(train_err))
    plt.plot(list(train_err.keys()), list(train_err.values()), label='train error')
    plt.plot(list(test_err.keys()), list(test_err.values()), label='test error')
    plt.legend(loc='upper right')
    plt.show()

In [9]:
def gb_fit(n_trees, max_depth, x_train, x_test, y_train, y_test, eta):

    # Деревья будем записывать в список
    trees = []

    # Будем записывать ошибки на обучающей и тестовой выборке на каждой итерации в список
    train_errors = []
    test_errors = []

    for i in range(n_trees):
        tree = DecisionTreeRegressor(max_depth=max_depth, random_state=42)

        # первый алгоритм просто обучаем на выборке и добавляем в список
        if len(trees) == 0:
            # обучаем первое дерево на обучающей выборке
            tree.fit(x_train, y_train)

            train_errors.append(mean_squared_error(y_train, gb_predict(x_train, trees, eta)))
            test_errors.append(mean_squared_error(y_test, gb_predict(x_test, trees, eta)))
        else:
            # Получим ответы на текущей композиции
            target = gb_predict(x_train, trees, eta)

            # алгоритмы начиная со второго обучаем на сдвиг
            tree.fit(x_train, residual(y_train, target))

            train_errors.append(mean_squared_error(y_train, gb_predict(x_train, trees, eta)))
            test_errors.append(mean_squared_error(y_test, gb_predict(x_test, trees, eta)))

        trees.append(tree)

    return trees, train_errors, test_errors

def calc_gb_fit_evaluate_alg(tree, depth):
    eta = 0.1
    trees, train_errors, test_errors = gb_fit(tree, depth, X_train, X_test, y_train, y_test, eta)
    train_error, test_error = evaluate_alg(X_train, X_test, y_train, y_test, trees, eta)
    print(f"tree={tree} depth={depth}")
    return train_error, test_error, tree, depth


In [10]:
from multiprocessing import Process, Value, Array


xxx = []
tree_count = 30
depth_count = 15

def multi_run_wrapper(args):
    return calc_gb_fit_evaluate_alg(*args)

for x, y in np.ndindex((tree_count, depth_count)):
    xxx.append((x +1, y + 1))

if __name__ == "__main__":
    from multiprocessing import Pool

    pool = Pool(20)
    result = pool.map(multi_run_wrapper,xxx)
    print(result)


tree=1 depth=7tree=1 depth=1tree=1 depth=13


tree=1 depth=14
tree=1 depth=2tree=1 depth=8

tree=1 depth=15
tree=1 depth=3tree=1 depth=9

tree=2 depth=10
tree=2 depth=4
tree=1 depth=4
tree=1 depth=10
tree=1 depth=5
tree=1 depth=11
tree=1 depth=12tree=2 depth=1

tree=1 depth=6
tree=3 depth=1
tree=2 depth=5tree=3 depth=13

tree=2 depth=11
tree=3 depth=7
tree=2 depth=2
tree=2 depth=6tree=2 depth=12

tree=4 depth=10
tree=4 depth=4
tree=2 depth=3
tree=2 depth=7
tree=2 depth=13
tree=3 depth=2
tree=3 depth=14
tree=3 depth=8
tree=2 depth=8
tree=2 depth=14
tree=5 depth=7
tree=5 depth=1
tree=5 depth=13
tree=2 depth=15
tree=2 depth=9
tree=3 depth=3
tree=3 depth=15
tree=3 depth=9
tree=4 depth=5
tree=4 depth=11
tree=6 depth=4
tree=6 depth=10
tree=3 depth=4
tree=3 depth=10
tree=4 depth=1
tree=3 depth=5
tree=7 depth=1
tree=7 depth=13
tree=3 depth=11
tree=7 depth=7tree=4 depth=6

tree=5 depth=2tree=4 depth=12
tree=5 depth=14

tree=5 depth=8
tree=3 depth=6
tree=3 depth=12
tree=4 depth=2
tree=8 depth=4


In [11]:
from matplotlib import cm
from matplotlib import interactive
import ipympl
interactive(True)
%matplotlib widget

mse_train_line, mse_test_line, trees_line, depth_line = zip(*result)

fig = plt.figure()

ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(np.reshape(trees_line, (tree_count, depth_count)), np.reshape(depth_line, (tree_count, depth_count)), np.reshape(mse_train_line, (tree_count, depth_count)), cmap="Blues")

ax.plot_surface(np.reshape(trees_line, (tree_count, depth_count)), np.reshape(depth_line, (tree_count, depth_count)), np.reshape(mse_test_line, (tree_count, depth_count)), cmap="Reds")

ax.set_xlabel('X trees')
ax.set_ylabel('Y depths')
ax.set_zlabel('Z mse')

plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Вывод: при глубине большей 2 начинается переобучение модели и падения качества, при увеличение количества деревьев точность модели увеличивается, но при при колличестве деревьев больше 20 не дает значимого прироста точности.